In [2]:
'''This script train a model to perform number classification into 4 classes.
CNN was picked to performing trainging since they can be scaled to higer resolutions too
and most importantly have a higher accuracy than their counterparts.'''
import pickle
import tensorflow as tf
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
'''First Intitutions: Unpickle the test/test image objects and save them
locally for further training of ML or DL Model'''
label_pickle = open("train_label.pkl","rb")
labels=pickle.load(label_pickle)
with open('test_image.pkl','rb') as file:
    test_image=pickle.load(file)

with open('train_image.pkl','rb') as file:
    train_image=pickle.load(file)
#print(type(test_image))
#print(type(train_image))    
#print(type(labels))
'''On examining the unpickled objects, it was known that their types were lists
'''
#print(len(test_image))
#print(len(train_image))    
#print(len(labels))
#On examining the length it was known that there were 8000 samples of numbers images
#print(test_image)
# Each element in the list had 784 items, hence after looking upthe internet it was known 
#that the values were 0 to 255 grey scale.
# By intuition we could imagine a square pixel image of 28*28 pixel image 
#By taking this intiution forward , we proceed to reshape the np array into 28,28 to 
#eventually feed into out CNN model for training images.

test_image_np=np.array(test_image)
train_image_np=np.array(train_image)
#The lists are converted into np array for test train splitting and entutally input into 
# the models
# print(test_image.shape)
train_shuffle,labels_shuffle = shuffle(train_image_np, labels, random_state=42)
train_shuffle,labels_shuffle = shuffle(train_shuffle, labels_shuffle, random_state=38)
'''provided train data is shuffled for better training and accuracy by sklearn's shuffle method.
We then spilit the train and test images into 9:1 ratio for optimum model accurracy 
as the sample size of the dataset is less'''

x_train=np.array(train_shuffle[0:7200])
y_train=np.array(labels_shuffle[0:7200])
x_test=np.array(train_shuffle[7200:8000])
y_test=np.array(labels_shuffle[7200:8000])
x_train_al=x_train
y_train_al=y_train
x_train.shape
# test_image_np = test_image_np.reshape(x_train.shape[0], 28, 28, 1)

(7200, 784)

In [3]:
'''In Keras API, we need 4-dims numpy arrays but our array is 3-dimensional.
In addition, we must normalize our data as required in neural network models by diving its 
float value by the maxaimum RGB value(grey scale) by its maximum value 255'''
# Reshaping the array to 4-dims so that it can work with the Keras API

x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
#reshaping the np arrays
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

input_shape = (28, 28, 1)

# Making sure that the values are float so that we can get decimal points after division

x_train = x_train.astype('float32')

x_test = x_test.astype('float32')

# Normalizing the RGB codes by dividing it to the max RGB value.

x_train /= 255

x_test /= 255

print('x_train shape:', x_train.shape)

print('Number of images in x_train', x_train.shape[0])

print('Number of images in x_test', x_test.shape[0])

x_train shape: (7200, 28, 28, 1)
Number of images in x_train 7200
Number of images in x_test 800


In [4]:
#Building the CNN
'''We will build our model by using high level Keras API 
which uses either TensorFlow or Theano on the backend'''

from keras.models import Sequential

from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D

# Creating a Sequential Model and adding the layers

model = Sequential()
'''Convolutional layer preserve the relationship between different parts of an image,
filtering the image with a smaller pixel filter 
to decrease the size of the image without loosing the relationship between pixels.'''
model.add(Conv2D(28, kernel_size=(3,3), input_shape=input_shape))
#  Pooling layer reduce the spatial size of the representation to reduce the parameter
# counts which reduces the computational complexity
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten()) # Flattening the 2D arrays for fully connected layers

model.add(Dense(128, activation=tf.nn.relu))

model.add(Dropout(0.2))
#In addition, Dropout layers fight with the overfitting by disregarding some of the neurons while training while 
# Flatten layers flatten 2D arrays to 1D array before building the fully connected layers.
model.add(Dense(10,activation=tf.nn.softmax))


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Using TensorFlow backend.


In [5]:
#  We have to optimize with a given loss function which uses a metric. Then, we can fit the model
# by using the train data.
model.compile(optimizer='adam', 

              loss='sparse_categorical_crossentropy', 

              metrics=['accuracy'])

model.fit(x=x_train,y=y_train, epochs=12)

Instructions for updating:
Use tf.cast instead.
Epoch 1/12
7200/7200 [==============================] - 9s 1ms/step - loss: 0.6413 - acc: 0.7460
Epoch 2/12
7200/7200 [==============================] - 9s 1ms/step - loss: 0.4552 - acc: 0.8243
Epoch 3/12
7200/7200 [==============================] - 9s 1ms/step - loss: 0.3934 - acc: 0.8479
Epoch 4/12
7200/7200 [==============================] - 9s 1ms/step - loss: 0.3390 - acc: 0.8686
Epoch 5/12
7200/7200 [==============================] - 9s 1ms/step - loss: 0.3061 - acc: 0.8800
Epoch 6/12
7200/7200 [==============================] - 9s 1ms/step - loss: 0.2579 - acc: 0.9001
Epoch 7/12
7200/7200 [==============================] - 9s 1ms/step - loss: 0.2320 - acc: 0.9099
Epoch 8/12
7200/7200 [==============================] - 9s 1ms/step - loss: 0.1984 - acc: 0.9269
Epoch 9/12
7200/7200 [==============================] - 9s 1ms/step - loss: 0.1677 - acc: 0.9378
Epoch 10/12
7200/7200 [==============================] - 9s 1ms/step - loss: 0.

In [6]:
model.evaluate(x_test, y_test)
#We finally evalaute the model to chcck for its accuracy against out test data and labels provided

800/800 [==============================] - 1s 807us/step


[0.41383938074111937, 0.865]

In [7]:
#Sample prediction with test data 
image_index = 266
pred = model.predict(x_test[image_index].reshape(1, 28, 28, 1))
print(pred.argmax())
print(y_test[image_index])

3
3


In [8]:
'''This script will predict the numbers into their respective classes and then add them into 
a csv file.'''
import csv

# csvData = [['Person', 'Age'], ['Peter', '22'], ['Jasmine', '21'], ['Sam', '24']]
csvdata=[['image_index','class']]
#Initial header of the csv file
'''the list items containing the test_image_index and the predicted class for 
the reshaped test_image_np is appened onyto the csvdata list'''
for image_index in range(0,len(test_image_np)):
    csvdata.append([image_index,model.predict(test_image_np[image_index].reshape(1, 28, 28, 1)).argmax()])
    
with open('RAM PRASAD E NAIK.csv', 'w',newline='') as csvFile:        
    writer = csv.writer(csvFile)
    writer.writerows(csvdata)
csvFile.close()
'''A new file is opened and csv writer() is used to write the rows of data held by the csvdata 
list'''

'A new file is opened and csv writer() is used to write the rows of data held by the csvdata \nlist'